Концепция word2vec: каждому слову сопоставляется вектор вещественных чисел. Чем больше косинусная близость между двумя векторами (т.е. косинус угла между ними), тем ближе эти слова друг к другу по значению. Такие модели обучаются на больших корпусах текстов.

https://rusvectores.org/ru/models/

Задание для самостоятельного выполнения в классе (чтобы не скучать): напишите функцию, которая вычисляет косинусную близость между двумя векторами (списками) по формуле:

$$cos(\theta) = \frac{A \cdot B}{\lVert A \rVert \lVert B \rVert} = \frac{\sum_{i=1}^{n}{A_{i} B_{i}}}{\sqrt{\sum_{i=1}^{n}{A_{i}^{2}}}\sqrt{\sum_{i=1}^{n}{B_{i}^{2}}}}$$

Используйте только встроенные функции!

In [5]:
def cossim(a: list[float], b: list[float]) -> float:
    pass

In [17]:
a = [1, 0, 0]
b = [0, 1, 0]

То же самое в numpy и scipy:

In [ ]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(a, b)/(norm(a)*norm(b))
print(cos_sim)

In [ ]:
from scipy.spatial.distance import cosine

cos_sim = 1 - cosine(a, b)
print(cos_sim)

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/220.zip
!unzip 220.zip

Библиотека gensim:

In [ ]:
from gensim.models import KeyedVectors

https://radimrehurek.com/gensim/models/keyedvectors.html

In [ ]:
model = KeyedVectors.load_word2vec_format("model.bin", binary=True)

In [ ]:
model.most_similar("фонетика_NOUN")

Для поиска по коллекции документов можно использовать и этот способ векторного представления: представим каждый текст как среднее значение векторов для входящих в него слов: https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.get_mean_vector

In [ ]:
sentence = ["фонетика_NOUN"]
sentence_embedding = model.get_mean_vector(sentence, ignore_missing=True)

Задание для совместного выполнения (чтобы не скучать): давайте обучим свою модель на тезисах студенческой конференции. Для этого:
1. Каждый текст разобьём на предложения
2. Каждое предложение разобьём на слова
3. Каждое слово приведём к нормальной форме и к нижнему регистру
4. Создадим большой список всех предложений
5. Обучим модель:

In [ ]:
!wget pkholyavin.github.io/year4programming/conference_stud_clean.pkl

In [ ]:
from gensim.models import Word2Vec

In [ ]:
corpus = []
model_conf = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

Наши вектора хранятся в атрибуте wv:

In [ ]:
model_conf.wv.most_similar("фонетика")

Задание на дом:

Создайте систему поиска по текстам, которая по функционалу аналогична предыдущей, но в качестве функции ранжирования использует косинусную близость между вектором текста и вектором запроса:

https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.n_similarity

In [ ]:
model_conf.wv.n_similarity(["фонетика"], ["фонетика", "и", "фонология"])

Используйте модель с сайта rusvectores. Обратите внимание, что у каждого слова есть частеречный тег, при этом части речи отличаются от тех, которые даёт pymorphy2:

https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

https://universaldependencies.org/u/pos/all.html

In [ ]:
set(w.split("_")[-1] for w in model.key_to_index)

Таким образом, вам нужно:
1. Для каждого слова в текстах и в поисковом запросе определить частеречную принадлежность (это легко сделать вместе с лемматизацией)
2. Определить, какой тег нужно присвоить слову

Замечание: если мы будем использовать `n_similarity`, то тогда вектор каждого текста будет вычисляться заново при каждом новом поиске. Кажется, это неоптимально; поэтому хорошей оптимизацией будет:
1) вычислить вектора текстов заранее с помощью `get_mean_vector`
2) вычислить вектор запроса с помощью этого же метода
2) считать косинусную близость вручную с помощью numpy или scipy.